In [ ]:
%matplotlib inline
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoAlertPresentException
import sys

import unittest, time, re
from bs4 import BeautifulSoup as bs
from dateutil import parser
import pandas as pd
import itertools
import matplotlib.pyplot as plt

In [95]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

option = webdriver.ChromeOptions()
option.add_argument("--incognito")


driver = webdriver.Chrome("D:/chromedriver_win32/chromedriver.exe",chrome_options=option)
driver.get('https://twitter.com/maggiew61/following')


In [45]:
for i in range(1,8):  #execute sth a number of times,starting from 1 and end at 7
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)
    print(i)

1
2
3
4
5
6
7


In [85]:
import pandas as pd
from bs4 import BeautifulSoup
# html_source = driver.page_source
# sourcedata= html_source.encode('utf-8')
# soup=bs(sourcedata)
soup = BeautifulSoup(driver.page_source ,"html.parser")
#find the attribute 
arr = [x.div['data-screen-name'] for x in soup.body.find_all('div', attrs={'data-item-type':'user'})]
print('arr is',arr)
bios = [x.p.text for x in soup.body.findAll('div', attrs={'data-item-type':'user'})]
print('bios is',bios)
fullnames = [x.text.strip() for x in soup.body.findAll('a', 'fullname')][1:] # avoid your own name
d = {'usernames': arr, 'bios': bios, 'fullnames': fullnames}
df = pd.DataFrame(data=d)
df.to_csv('BASICDATA2.csv')

arr is ['robkardashian', 'KylieJenner', 'KrisJenner', 'khloekardashian', 'KendallJenner', 'kourtneykardash', 'KimKardashian', 'jimmykimmel', 'Oprah', 'TheEllenShow', 'ddlovato', 'selenagomez', 'csa_asc', 'ExploreCanada', 'Canada', 'realDonaldTrump', 'AShinOfficial', 'Mayday_EN']
bios is ['snapchat:  robphuckedme', 'http://kyliecosmetics.com\xa0 http://KylieJennerShop.com\xa0', 'MOM, MANAGER, MOMAGER, LOVER OF LIFE, LOVER OF CHRIST', 'Get my official app #unlockmyworld http://AppStore.com/KhloeKardashian\xa0', '', '', '', '1997 Bowling Digest Co-Bowler of the Year  http://Facebook.com/JimmyKimmel\xa0', '', 'Comedian, talk show host and ice road trucker. My tweets are real, and they’re spectacular.', 'Singer, Songwriter, Actress, Entrepreneur, Philanthropist  #Sober: http://demilovato.co/sober\xa0', "Get 'Back To You' out now: http://smarturl.it/backtoyousg\xa0", 'The Canadian Space Agency coordinates all civil, space-related programs on behalf of the Government of Canada. Terms: http://

In [102]:
from time import sleep
df = pd.read_csv('BASICDATA2.csv', encoding = "ISO-8859-1")
# arr = df.usernames
arr = [x.div['data-screen-name'] for x in soup.body.find_all('div', attrs={'data-item-type':'user'})]
def getTimestamps(x):
    temp = x.findAll('span', '_timestamp')
    if len(temp) > 0:
        return temp[0].get('data-time')
    else:
        return None
    
# now get the user's own timeline
for i in range(0,len(arr)):
    currentUser = arr[i]
    print('doing user:' + str(i) + ' ' + currentUser)
#     driver.base_url = "https://twitter.com/" + currentUser + '/with_replies'
    
    #test
# https://sqa.stackexchange.com/questions/7178/selecting-xpath-using-a-variable-in-selenium-2
    driver.find_element_by_xpath("//a[contains(.,'" + currentUser + "')]").click()
#     soup = BeautifulSoup(driver.page_source ,"html.parser")  #use soup to parse the page
    #test ends
    
#     driver.get(driver.base_url)
    html_source = driver.page_source
    dailyemail_links = html_source.encode('utf-8')
    soup=bs(dailyemail_links, "lxml")
    time.sleep(2)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)
    # name
    name = soup.find('a', "ProfileHeaderCard-nameLink").text
    # loc
    temp = soup.find('span', 'ProfileHeaderCard-locationText')
    temp = temp.text if temp else ''
    loc = temp.strip() if temp else ''
    # url
    temp = soup.find('span', 'ProfileHeaderCard-urlText')
    temp = temp.a if temp else None
    temp2 = temp.get('title') if temp else None
    url = temp2 if temp2 else (temp.get('href') if temp else None)
    # stats
    temp = soup.find('a',{'data-nav': 'tweets'})
    stats_tweets = temp.find('span', 'ProfileNav-value')['data-count'] if temp else 0
    temp = soup.find('a',{'data-nav': 'following'})
    stats_following = temp.find('span', 'ProfileNav-value')['data-count'] if temp else 0
    temp = soup.find('a',{'data-nav': 'followers'})
    stats_followers = temp.find('span', 'ProfileNav-value')['data-count'] if temp else 0
    temp = soup.find('a',{'data-nav': 'favorites'})
    stats_favorites = temp.find('span', 'ProfileNav-value')['data-count'] if temp else 0
    # all text
    text = [''.join(x.findAll(text=True)) for x in soup.body.findAll('p', 'tweet-text')]
    # most recent activity
    alltweets = soup.body.findAll('li', attrs={'data-item-type':'tweet'})
    tweetTimestamps = list(map(getTimestamps, alltweets)) if len(alltweets) > 0 else 0
    # engagements
    noretweets = [x.findAll('span', 'ProfileTweet-actionCount') for x in alltweets if not x.div.get('data-retweet-id')]
    templist = [x.findAll('span', 'ProfileTweet-actionCount') for x in alltweets if not x.div.get('data-retweet-id')]
    templist = [item for sublist in templist for item in sublist]
    engagements = sum([int(x.get('data-tweet-stat-count')) for x in templist if x.get('data-tweet-stat-count')])
    main = pd.concat([main, pd.DataFrame(data = {
        'user': [currentUser],
        'text': [text],
        'mostrecentTimestamp': [tweetTimestamps],
        'engagements': [engagements],
        'name': [name],
        'loc': [loc],
        'url': [url],
        'stats_tweets': [stats_tweets],
        'stats_following': [stats_following],
        'stats_followers': [stats_followers],
        'stats_favorites': [stats_favorites],
    })])
    driver.back()
    sleep(5)
#     main.to_csv('BASICDATA2.csv')

doing user:0 robkardashian
doing user:1 KylieJenner
doing user:2 KrisJenner
doing user:3 khloekardashian
doing user:4 KendallJenner
doing user:5 kourtneykardash
doing user:6 KimKardashian
doing user:7 jimmykimmel
doing user:8 Oprah
doing user:9 TheEllenShow
doing user:10 ddlovato
doing user:11 selenagomez
doing user:12 csa_asc
doing user:13 ExploreCanada
doing user:14 Canada


WebDriverException: Message: chrome not reachable
  (Session info: chrome=67.0.3396.99)
  (Driver info: chromedriver=2.40.565498 (ea082db3280dd6843ebfb08a625e3eb905c4f5ab),platform=Windows NT 10.0.17134 x86_64)
